In [16]:
import pandas as pd


import warnings
warnings.filterwarnings('ignore')  

# Load the movies_metadata dataset
movies_metadata = pd.read_csv('Data/movies_metadata.csv')

# Load the keywords dataset
keywords = pd.read_csv('Data/keywords.csv')

# Convert the 'id' column to int64
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')

# Merge the datasets    
movies = pd.merge(movies_metadata, keywords, on='id', how='inner')

movies.head()



,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
movie_overview = movies['overview'].fillna('')
movie_title = movies['original_title'].fillna('')
movie_index = pd.Series(movies.index, index=movies['original_title']).drop_duplicates()
tfdif = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfdif.fit_transform(movie_overview)
movie_index

original_title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
رگ خواب                        46477
Siglo ng Pagluluwal            46478
Betrayal                       46479
Satana likuyushchiy            46480
Queerama                       46481
Length: 46482, dtype: int64

In [24]:
title = 'Furious 7'
idx = movie_index[title]
movie_text_summary = movies['overview'][idx]
movie_test_matrix = tfdif.transform([movie_text_summary])
print(movie_test_matrix.shape)


(1, 75827)


In [27]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(movie_test_matrix, tfidf_matrix).tolist()[0] 
print(len(cosine_sim))



46482


In [30]:
sim_scores = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True )
print(sim_scores[:5])

[(28902, 1.0), (4255, 0.28472924118598303), (21047, 0.22756352667706278), (42013, 0.21858603856946132), (17123, 0.1958027781351207)]


In [31]:
movie_indices = [i[0] for i in sim_scores[:5]]
print(movies['original_title'].iloc[movie_indices])

28902                   Furious 7
4255     The Fast and the Furious
21047            Fast & Furious 6
42013              Los violadores
17123                   Fast Five
Name: original_title, dtype: object
